In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List
from torch.utils.data import Sampler

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [3]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.6.0].
device:[cuda:0].


## Training


In [4]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [41]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [43]:
# Preprocess

import re
def preprocess(text):
    text = re.sub(r"\\r|\\n|\n|\\t", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

train_dataset = pd.DataFrame(train_dataset)
train_dataset['context'] = train_dataset['context'].apply(lambda x: preprocess(x))
train_dataset = Dataset.from_pandas(train_dataset)

In [6]:
class CustomSampler(Sampler) :
    def __init__(self, data_source, batch_size) :
        self.data_source = data_source
        self.batch_size = batch_size

    def __iter__(self) :
        n = len(self.data_source)
        index_list = []
        while True :
            out = True
            for i in range(self.batch_size) :
                tmp_data = random.randint(0, n-1)
                index_list.append(tmp_data)
            for f, s in zip(index_list, index_list[1:]) :
                if abs(s-f) <= 2 :
                    out = False
            if out == True :
                break

        while True : # 추가 삽입
            tmp_data = random.randint(0, n-1)
            if (tmp_data not in index_list) and \
                (abs(tmp_data-index_list[-i]) > 2 for i in range(1,self.batch_size+1)) \
            : 
                index_list.append(tmp_data)
            if len(index_list) == n :
                break
        return iter(index_list)

    def __len__(self) :
        return len(self.data_source)

In [7]:
# Anwer
class DenseRetrieval:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        p_encoder,
        q_encoder,
        sampler
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder
        self.sampler = sampler

    def train(self, args=None, tokenizer = None):
        if args is None:
            args = self.args
        if tokenizer is None :
            tokenizer = self.tokenizer

        for i in (range(len(self.dataset))) :
            if i == 0 :
                q_seqs = tokenizer(
                    self.dataset[i]['question'],
                    padding='max_length',
                    truncation = True,
                    return_tensors='pt'
                )
                p_seqs = tokenizer(
                    self.dataset[i]['context'],
                    truncation = True,
                    stride = 128,
                    padding='max_length',
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    return_tensors='pt'
                )

                p_seqs.pop('overflow_to_sample_mapping')
                p_seqs.pop('offset_mapping')

                for k in q_seqs.keys() :
                    q_seqs[k] = q_seqs[k].tolist()
                    p_seqs[k] = p_seqs[k].tolist()
            else :
                tmp_q_seq = tokenizer(
                    self.dataset[i]['question'],
                    padding='max_length',
                    truncation = True,
                    return_tensors='pt')
                tmp_p_seq = tokenizer(
                    self.dataset[i]['context'],
                    truncation = True,
                    stride = 128,
                    padding='max_length',
                    return_overflowing_tokens=True,
                    return_offsets_mapping=True,
                    return_tensors='pt'
                )

                tmp_p_seq.pop('overflow_to_sample_mapping')
                tmp_p_seq.pop('offset_mapping')

                for k in tmp_p_seq.keys() :
                    tmp_p_seq[k] = tmp_p_seq[k].tolist()
                    tmp_q_seq[k] = tmp_q_seq[k].tolist()


                for j in range(len(tmp_p_seq['input_ids'])) :
                    q_seqs['input_ids'].append(tmp_q_seq['input_ids'][0])
                    q_seqs['token_type_ids'].append(tmp_q_seq['token_type_ids'][0])
                    q_seqs['attention_mask'].append(tmp_q_seq['attention_mask'][0])
                    p_seqs['input_ids'].append(tmp_p_seq['input_ids'][j])
                    p_seqs['token_type_ids'].append(tmp_p_seq['token_type_ids'][j])
                    p_seqs['attention_mask'].append(tmp_p_seq['attention_mask'][j])

        for k in q_seqs.keys() :
            q_seqs[k] = torch.tensor(q_seqs[k])
            p_seqs[k] = torch.tensor(p_seqs[k])

        train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])
        sampler = self.sampler(train_dataset, args.per_device_train_batch_size)
        train_dataloader = DataLoader(train_dataset,
                                      batch_size=args.per_device_train_batch_size,
                                      sampler = sampler)

        no_decay = ["bias" ,"LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            # eps=args.adam_epsilon
        )

        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
        
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        # torch.cuda.empty_cache()

        train_iterator = trange(int(args.num_train_epochs), desc="Epoch")
        self.q_encoder.train()
        self.p_encoder.train()
        for epoch, _ in enumerate(train_iterator):
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            loss_value=0 # Accumulation할 때 진행
            losses = 0
            for step, batch in enumerate(epoch_iterator):
                if torch.cuda.is_available():
                    batch = tuple(t.cuda() for t in batch)

                p_inputs = {'input_ids': batch[0],
                            'attention_mask': batch[1],
                            'token_type_ids': batch[2]
                            }
                
                q_inputs = {'input_ids': batch[3],
                            'attention_mask': batch[4],
                            'token_type_ids': batch[5]}

                p_outputs = self.p_encoder(**p_inputs)  # (batch_size, emb_dim)
                q_outputs = self.q_encoder(**q_inputs)  # (batch_size, emb_dim)

                # Calculate similarity score & loss
                sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

                # target: position of positive samples = diagonal element 
                # targets = torch.arange(0, args.per_device_train_batch_size).long()
                targets = torch.arange(0, len(p_inputs['input_ids'])).long()
                
                if torch.cuda.is_available():
                    targets = targets.to('cuda')
                    
                sim_scores = F.log_softmax(sim_scores, dim=1)

                loss = F.nll_loss(sim_scores, targets)
                losses += loss.item()
                if step % 100 == 0 :
                    print(f'{epoch}epoch loss: {losses/(step+1)}') # Accumulation할 경우 주석처리

                # ################ACCUMULATION###############################
                # if (step+1) % args.gradient_accumulation_steps == 0 :
                #     optimizer.step()
                #     scheduler.step()
                #     self.q_encoder.zero_grad()
                #     self.p_encoder.zero_grad()

                # losses += loss.item()
                # if (step+1) % 64 == 0:
                #     train_loss = losses / 64
                #     print(f'training loss: {train_loss:4.4}')
                #     losses = 0
                # ###########################################################
                self.q_encoder.zero_grad()
                self.p_encoder.zero_grad()
                loss.backward()
                optimizer.step()
                scheduler.step()

                # global_step += 1
                
                #torch.cuda.empty_cache()
                del p_inputs, q_inputs

        return self.p_encoder, self.q_encoder

In [9]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=1,
    num_train_epochs=10,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# num_added_toks = tokenizer.add_tokens(['\\n']) # 32000번째로 삽입
p_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)
q_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

In [11]:
# sampler = CustomSampler()
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder,
    sampler = CustomSampler
)
p_encoder, q_encoder = retriever.train()

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

0epoch loss: 27.258563995361328
0epoch loss: 3.188428820009426


In [ ]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [ ]:
# p_encoder = retriever.p_encoder
# q_encoder = retriever.q_encoder
with torch.no_grad() :
    p_encoder.eval()

    p_embs = []
    for p in tqdm(corpus) :
        p = tokenizer(p, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
p_embs = torch.Tensor(p_embs).squeeze()

In [ ]:
import pickle

file_path = '/opt/ml/custom/passage_embedding_special_shuffle_100.bin'
with open(file_path, 'wb') as file :
    pickle.dump(p_embs, file)

In [ ]:
p_encoder.cpu()
del p_encoder
torch.cuda.empty_cache()


In [ ]:
torch.save(q_encoder, '/opt/ml/custom/q_encoder_special_shuffle_100.pt')

## Get Relavant Documnet

In [ ]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [ ]:
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import pickle
with open('/opt/ml/custom/passage_embedding_special_shuffle_100.bin', 'rb') as file :
    p_embs = pickle.load(file)
p_embs = p_embs

q_encoder = torch.load('/opt/ml/custom/q_encoder_special_shuffle_100.pt')

In [ ]:
def get_relavant_doc(queries, q_encoder, p_embs, k=1) :
    with torch.no_grad() :
        q_encoder.eval()
        q_seqs_val = tokenizer(queries, padding='max_length',truncation=True,return_tensors='pt').to(device)
        q_emb = q_encoder(**q_seqs_val).to('cpu')
    dot_prod_scores = torch.mm(q_emb, p_embs.T)
    sort_result = torch.sort(dot_prod_scores, dim=1, descending=True)

    scores, ranks = sort_result[0], sort_result[1]

    result_scores = []
    result_indices = []
    for i in range(len(ranks)) :
        result_scores.append(scores[i].tolist()[:k])
        result_indices.append(ranks[i].tolist()[:k])
    
    return result_scores, result_indices

In [61]:
def preprocess(text):
    text = re.sub(r"\\r|\\n|\n|\\t", " ", text)
    text = re.sub(r"\\", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text
    
def preprocess_train_val(train_val: dict) -> dict:
    """train_val such as datasets['train'] or datasets['validation'] from huggingface dataset"""
    # fetch from train_val datasets
    answer_start = train_val["answers"]["answer_start"][0]
    answer_text = train_val["answers"]["text"][0]
    context = train_val["context"]

    # slice the context
    preceding_text = context[:answer_start]
    following_text = context[answer_start + len(answer_text) :]

    # apply preprocessing for each sliced context
    preprocessed_preceding = preprocess(preceding_text)
    preprocessed_following = preprocess(following_text)

    # make preprocessed dataset dictionary
    new_context = preprocessed_preceding + answer_text + preprocessed_following
    new_answer_start = answer_start - abs(len(preceding_text) - len(preprocessed_preceding))

    # assign preprocessed values
    train_val["context"] = new_context
    train_val["answers"]["answer_start"][0] = new_answer_start

    return train_val

In [63]:
tmp_data = pd.DataFrame(dataset['validation'])

In [64]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
valid_dataset = dataset['validation']

In [ ]:
answer_start = train_val['answers']

In [67]:
preprocess_train_val(valid_dataset)

TypeError: list indices must be integers or slices, not str

In [ ]:
for i in range(len(tmp_data)) :
    